In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras
from tqdm import tqdm
import pandas as pd
import pickle
from time import time
#from keras.utils.vis_utils import plot_model

c:\python3.5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
keras.__version__

'2.1.6'

In [4]:
# A fast and memory efficient implementation
# by Hjelmqvist, Sten
def levenshtein(s, t):
    # degenerate cases
    if s == t:
        return 0
    if len(s) == 0:
        return len(t)
    if len(t) == 0:
        return len(s)
  
    # create two work vectors of integer distances
    #int[] v0 = new int[t.Length + 1];
    #int[] v1 = new int[t.Length + 1];
    v0 = []
    v1 = []
  
    # initialize v0 (the previous row of distances)
    # this row is A[0][i]: edit distance for an empty s
    # the distance is just the number of characters to delete from t
    # for (int i = 0; i < v0.Length; i++)
    # v0[i] = i;
    for i in range(len(t)+1):
        v0.append(i)
        v1.append(0)
 
    for i in range(len(s)): 
        # calculate v1 (current row distances) from the previous row v0
        # first element of v1 is A[i+1][0]
        # edit distance is delete (i+1) chars from s to match empty t
        v1[0] = i + 1
  
        # use formula to fill in the rest of the row
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1;
            v1[j + 1] = min(v1[j]+1, v0[j+1]+1, v0[j]+cost)
  
        # copy v1 (current row) to v0 (previous row) for next iteration
        for j in range(len(t)+1):
            v0[j] = v1[j]
  
    return v1[len(t)]

In [5]:
def vectorize(df):
    eng_sentences = list(df['x'])
    fra_sentences = list(df['y'])
    #print(eng_sentences[:5])
    nb_samples = df.shape[0]
    tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
    tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
    target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')
    # Vectorize the english and french sentences

    for i in tqdm(range(nb_samples)):
        for k,ch in enumerate(eng_sentences[i]):
            tokenized_eng_sentences[i,k,eng_char_to_index_dict[ch]] = 1

        for k,ch in enumerate(fra_sentences[i]):
            tokenized_fra_sentences[i,k,fra_char_to_index_dict[ch]] = 1

            # decoder_target_data will be ahead by one timestep and will not include the start character.
            if k > 0:
                target_data[i,k-1,fra_char_to_index_dict[ch]] = 1
    return tokenized_eng_sentences, tokenized_fra_sentences, target_data

In [6]:
def tokenize(seq):
    tokenized_eng_sentence = np.zeros(shape = (1,max_len_eng_sent,len(eng_chars)), dtype='float32')
    for k,ch in enumerate(seq):
        tokenized_eng_sentence[0,k,eng_char_to_index_dict[ch]] = 1
        
    return tokenized_eng_sentence


In [7]:
def decode_seq(inp_seq,encoder_model_inf,decoder_model_inf):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        #decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #predict_pr = decoder_model_inf.predict_proba(x=[target_seq] + states_val)
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #print(decoder_out)
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        #print('{} == {}'.format(sampled_fra_char,max_val))
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
    return translated_sent

In [8]:
def fitting(step, model, tokenized_eng_sentences, tokenized_fra_sentences, target_data):
    print('fitting on {} words...'.format(tokenized_eng_sentences.shape[0]))
    t = time()
    model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
              y=target_data,
              batch_size=64,
              epochs=40,
              validation_split=0.2,
             verbose = 0)
    #print(model.layers)
    print('creating new inference model...')
    # Encoder inference model
    encoder_model_inf = Model(encoder_input, encoder_states)
    # Decoder inference model
    decoder_state_input_h = Input(shape=(256,))
    decoder_state_input_c = Input(shape=(256,))
    decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

    decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                     initial_state=decoder_input_states)

    decoder_states = [decoder_h , decoder_c]

    decoder_out = decoder_dense(decoder_out)

    decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                              outputs=[decoder_out] + decoder_states )
    print('saving inference models...')
    encoder_model_inf.save('retraining_models/lev/encoder-{}.h5'.format(step))
    decoder_model_inf.save('retraining_models/lev/decoder-{}.h5'.format(step))
    model.save('retraining_models/lev/model-{}.h5'.format(step))
    print('fitting took {} min'.format(round(((time()-t)/60),4)))
    return(encoder_model_inf,decoder_model_inf)

In [9]:
#на n-ной итерации берем n*10000 примеров из теста, из них добавляем 10000 к трейну
def testing(encoder_model_inf,decoder_model_inf,train,train_rest,count,step):
    t = time()
    length = max((train_rest).shape[0],step*count) #сколько отрезаем для теста или берем весь
    test = train_rest[:length]
    print('testing on {} words...'.format(test.shape[0]))
    predictions = []
    dists = []
    for row in tqdm(range(test.shape[0])):
        x = test.iloc[row,5]
        x_seq = tokenize(x)
        y = decode_seq(x_seq,encoder_model_inf,decoder_model_inf)
        prediction = y.strip()
        predictions.append(prediction)
        true_form = test.iloc[row,4]
        dist = levenshtein(prediction,true_form)
        dists.append(dist)
    test['predictions'] = predictions
    test['dists'] = dists
    test = test.sort_values(by=['dists'], ascending = False)
    name = 'test_after_{}_step.csv'.format(step)
    test[['x','predictions','dists']].to_csv('training_loop/test/lev/' + name, sep='\t', encoding = 'utf-8')
    test = test[:count] # берем худшие count строк
    #выкидываем из train_rest строки, которые уходят в трейн
    valuelist = test['x']
    train_rest = train_rest[~train_rest.x.isin(valuelist)]
    train = test.copy()
    name_train = 'train_rest_after_{}_step.csv'.format(step)
    train_rest[['x']].to_csv('training_loop/test/lev/' + name_train, sep='\t', encoding = 'utf-8')
    train = test.copy()
    print('testing took {} min'.format(round(((time()-t)/60),4)))
    return train,train_rest
    

### один раз вычитываем данные,  создаем encoder decoder, компилируем модель

In [7]:
total_train = pd.read_csv('data/train/train_full.csv', sep='\t', encoding = 'utf-8')
total_train['x'] = total_train.apply(lambda row: row['lemma'] + str(row['formtag']), axis=1)
total_train['y'] ='\t' + total_train["form"] + '\n'

In [8]:
train_filtered = total_train[total_train.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)]

In [9]:
len(train_filtered)

509928

In [10]:
509928/12

42494.0

In [12]:
train_filtered[:5]

,lemma,gender,formtag,classtag,form,x,y
0,високос,м,2N,м1а,високосы,високос2N,\tвисокосы\n
1,зимовщик,мо,2N,мо3а,зимовщики,зимовщик2N,\tзимовщики\n
2,бензель,м,1D,м2а,бензелю,бензель1D,\tбензелю\n
3,хромоскоп,м,1A,м1а,хромоскоп,хромоскоп1A,\tхромоскоп\n
4,дерматология,ж,1L,ж7а,дерматологии,дерматология1L,\tдерматологии\n


In [13]:
data = pickle.load( open( "data.p", "rb" ))
eng_chars = data['eng_chars']
fra_chars = data['fra_chars']
max_len_eng_sent = data['max_len_eng_sent']
max_len_fra_sent = data['max_len_fra_sent']
eng_index_to_char_dict = data['eng_index_to_char_dict']
eng_char_to_index_dict = data['eng_char_to_index_dict']
fra_index_to_char_dict = data['fra_index_to_char_dict']
fra_char_to_index_dict = data['fra_char_to_index_dict']

In [14]:
# Encoder model
encoder_input = Input(shape=(None,len(eng_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]
# Decoder model
decoder_input = Input(shape=(None,len(fra_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])
# Compiling
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
count = 10000
train,train_rest = train_filtered[:count],train_filtered[count:]
step = 1
while train_rest.shape[0] > 0:
    print('iteration: {}, rest of train: {}'.format(step, train_rest.shape[0]))
    tokenized_eng_sentences, tokenized_fra_sentences, target_data = vectorize(train)
    encoder_model_inf,decoder_model_inf = fitting(step, model, tokenized_eng_sentences, tokenized_fra_sentences, target_data) 
    train,train_rest = testing(encoder_model_inf,decoder_model_inf,train,train_rest,count,step)
    step += 1

iteration: 1, rest of train: 373532


100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 49746.29it/s]


fitting on 10000 words...
creating new inference model...
saving inference models...
fitting took 9.6806 min
testing on 373532 words...


100%|████████████████████████████████| 373532/373532 [1:12:21<00:00, 86.03it/s]


testing took 72.4239 min
iteration: 2, rest of train: 363532


...

testing took 2.938 min
iteration: 38, rest of train: 3532


100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 44602.03it/s]


fitting on 10000 words...
creating new inference model...
saving inference models...
fitting took 10.5383 min
testing on 3532 words...


100%|██████████████████████████████████████| 3532/3532 [00:46<00:00, 76.31it/s]


testing took 0.7785 min


## testing Levenshtein

In [ ]:
data = pd.read_csv('data/eval/eval_test_full_letters.csv', sep='\t', encoding = 'utf-8')

In [ ]:
data = data[data.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)]

In [ ]:
data = data[['lemma','formtag','form','seq2seq_predictions']]

In [ ]:

dists = []
for row in tqdm(range(data.shape[0])):
    prediction = data.iloc[row,3]
    true_form = data.iloc[row,2]
    dist = levenshtein(prediction,true_form)
    dists.append(dist)
data['dist'] = dists

In [ ]:
data = data.sort_values(by=['dist'], ascending = False)

In [ ]:
#data.loc[data['form'] == 'ржанцы']

In [ ]:
data[:2000]